## [5주차] 기본과제 - 외부 블로그의 정보와 함께 챗봇 구현하기

#### lib download

In [1]:
!pip install langchain-community langchain-chroma langchain-openai bs4

#### 환경 설정

custom print : colab 화면을 작게 쓰면서 출력이 길게 되는 것 불편해서 생성

In [2]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from bs4 import SoupStrainer
import os

In [3]:
with open('secrets.env') as f:
    for line in f:
        k, v = line.strip().split("=")
        os.environ[k] = v

In [4]:
import textwrap

def custom_print(text, width=70):
    print(textwrap.fill(text, width=width))

In [34]:
from IPython.display import display, HTML

def display_prompt_flow(user_query, response_content):
    html = f"""
    <div style="font-family: monospace; line-height: 1.5;">
        <h4>🧑 User Query</h4>
        <pre style="white-space: pre-wrap; word-break: break-word;">{user_query}</pre>
        <hr>
        <h4>🤖 LLM Response</h4>
        <pre style="white-space: pre-wrap; word-break: break-word;">{response_content}</pre>
    </div>
    """
    display(HTML(html))


### RAG 구성

### 1. Document Loader   
* 웹문서 BeautifulSoup을 사용해서 html 안에 값 파싱하기   
* header 넣어주는 이유   
  사이트가 requests 같은 봇 요청을 차단할 수 있어서 브라우저에서 접근한 것처럼 보이게 하는 트릭   
* CustomWebBaseLoader   
  WebBaseLoader 사용 시 encoding 이슈로 text가 깨져보여 encoding 강제 지정하는 class 생성

In [5]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.1,
    top_p=0.9,
    max_tokens=1024,
    frequency_penalty=0.0
)

In [6]:
from langchain.document_loaders import WebBaseLoader
from langchain.schema import Document
import requests
from bs4 import BeautifulSoup, SoupStrainer

class CustomWebBaseLoader(WebBaseLoader):
    def _get_bs(self, url: str, **bs_kwargs) -> BeautifulSoup:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

        # 🔥 인코딩 강제 지정
        response.encoding = "utf-8"

        return BeautifulSoup(
            response.text,
            "html.parser",
            **bs_kwargs
        )

    def load(self) -> list[Document]:
        docs = []
        for url in self.web_paths:
            soup = self._get_bs(url, **(self.bs_kwargs or {}))
            text = soup.get_text(separator="\n").strip()
            metadata = {"source": url}
            docs.append(Document(page_content=text, metadata=metadata))
        return docs


In [7]:
from bs4 import SoupStrainer

loader = CustomWebBaseLoader(
    web_paths=["https://spartacodingclub.kr/blog/all-in-challenge_winner"],
    bs_kwargs={
        "parse_only": SoupStrainer("div", class_="editedContent")
    }
)

docs = loader.load()
print(docs[0].page_content[:500])
print(f"\n문서 길이: {len(docs[0].page_content)}")

코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.
<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.
🏆 대상
[Lexi Note] 언어공부 필기 웹 서비스
서비스 제작자: 다나와(김다애, 박나경)
💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷

문서 길이: 5462


### 2. Text Splitter
문서를 chunk로 나눌 수 있도록 해주는 부분

* CharacterTextSplitter   
단순한 문자 기반 분할기

* RecursiveCharacterTextSplitter (사용)   
우선 순위 separator list 기반 분할기




In [8]:
default_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
default_splits = default_text_splitter.split_documents(docs)
print(f"chunk 개수 : {len(default_splits)}")

chunk 개수 : 7


#### chunk split 어떻게 되었는지 확인해보기

In [9]:
import pandas as pd

d_df = pd.DataFrame([{
    "chunk_id": i,
    "content": chunk.page_content,
    "length": len(chunk.page_content),
    "source": chunk.metadata.get("source", "")
} for i, chunk in enumerate(default_splits)])

In [10]:
d_df

,chunk_id,content,length,source
0,0,코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비...,999,https://spartacodingclub.kr/blog/all-in-challe...
1,1,"사용한 기술 스택 \n-FE(프론트엔드): React, Tesseract.js, R...",896,https://spartacodingclub.kr/blog/all-in-challe...
2,2,"사용한 기술 스택 \n-FE(프론트엔드): React Native, Expo, Ax...",987,https://spartacodingclub.kr/blog/all-in-challe...
3,3,"사용한 기술 스택 \n-FE(프론트엔드): Flutter, Socket.IO, Ex...",975,https://spartacodingclub.kr/blog/all-in-challe...
4,4,💡A는 올해 복학한 3학년 학생입니다. 강의실과 도서관을 오가며 바쁜 일정을 소화하...,840,https://spartacodingclub.kr/blog/all-in-challe...
5,5,사용한 기술 스택 \n-FE(프론트엔드): Flutter\n-BE(백엔드): Fir...,956,https://spartacodingclub.kr/blog/all-in-challe...
6,6,"사용한 기술 스택\n-FE(프론트엔드): Spring Boot, MySQL, Spr...",545,https://spartacodingclub.kr/blog/all-in-challe...


In [11]:
print(d_df.loc[3, "content"])

사용한 기술 스택 
-FE(프론트엔드): Flutter, Socket.IO, Expo CLI, Axios, TanStack Query
-BE(백엔드): Spring Boot, Spring Security, JWT, MySQL, Spring WebSocket, AWS
🏅 입선
[Crewing] 연합동아리 정보 플랫폼
서비스 제작자: 동학대학운동(김민아, 임경진, 신은혜, 고수)
💡신입생인 S는 대학 생활을 알차게 보내고 싶어서 연합 동아리에 가입하려고 했지만 어떤 동아리가 자신에게 맞을지 고민이 많았습니다. 인터넷에 검색해보니 연합 동아리 후기는 대부분 여기저기 흩어져 있고, 신뢰할 만한 정보를 찾기 어려웠습니다.
<Crewing>은 대학생들이 다양한 연합 동아리에 쉽게 가입하고, 적절한 동아리를 찾을 수 있도록 지원해주는 아카이빙 플랫폼입니다. 회원가입 시 생년, 성별, 관심 분야를 입력하자 딱 맞는 동아리를 추천해주죠. 플랫폼 내에서 동아리의 리크루팅 과정인 서류 전형, 인터뷰 일정, 최종 결과 발표 등을 한 번에 해결할 수 있어요. 실제 동아리에 가입한 사람들의 솔직한 후기를 제공해주기 때문에 보다 정확한 정보를 얻을 수 있어요. Crewing은 신뢰할 수 있는 정보와 솔직한 후기를 제공해주기 때문에 효율적으로 내게 꼭 맞는 동아리를 선택할 수 있습니다.
사용한 기술 스택 
-FE(프론트엔드): Spring Boot, Redis, MySQL
-BE(백엔드): SwiftUI Framework, OAuth 2.0
🏅 입선
[학교생활 매니저] 학교생활 관리 서비스
서비스 제작자: 아이칼F4(조민제, 이민기, 강건, 박근우)
💡A는 올해 복학한 3학년 학생입니다. 강의실과 도서관을 오가며 바쁜 일정을 소화하느라 정신이 없지만, 수업마다 나오는 과제와 각종 활동, 시험 준비까지 겹치면서 혼란에 빠지기 일쑤였습니다. 복학생이다 보니 학교에서 전달되는 공지사항도 제대로 전달받지 못해 항상 중요한 정보를 놓칠까 봐 걱정이었죠.


데이터를 확인해보면 프로젝트별로 나뉜게 아니라 정말 text 길이에 따라 split된 걸 볼 수 있었다. RecursiveCharacterTextSplitter을 선언하기는 했지만, 실제 사용은 CharacterTextSplitter랑 동일하게 사용하고 있다.

따라서 우선순위를 주어서 내용별로 나눌 수 있도록 개선했다.

#### chunk split 개선
프로젝트 별로 앞에 이모티콘 "🏆","🎖️","🏅" 이 보이고 있다. 따라서 문단보다 해당 이모티콘으로 chunk를 나누도록 우선순위를 설정해주었다.

In [12]:
new_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["🏆","🎖️","🏅","\n\n", "\n", ".", " ", ""]
)
new_splits = new_text_splitter.split_documents(docs)
print(f"chunk 개수 : {len(new_splits)}")

chunk 개수 : 8


In [13]:
import pandas as pd

df = pd.DataFrame([{
    "chunk_id": i,
    "content": chunk.page_content,
    "length": len(chunk.page_content),
    "source": chunk.metadata.get("source", "")
} for i, chunk in enumerate(new_splits)])

df

,chunk_id,content,length,source
0,0,코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비...,309,https://spartacodingclub.kr/blog/all-in-challe...
1,1,🏆 대상\n[Lexi Note] 언어공부 필기 웹 서비스\n서비스 제작자: 다나와(...,795,https://spartacodingclub.kr/blog/all-in-challe...
2,2,🎖️ 우수상\n[우리집 히어로즈] 벌레 퇴치 영웅 매칭 서비스\n서비스 제작자: 인...,709,https://spartacodingclub.kr/blog/all-in-challe...
3,3,🎖️ 우수상\n[에코 클래스룸] 수업 실시간 소통 서비스\n서비스 제작자: This...,863,https://spartacodingclub.kr/blog/all-in-challe...
4,4,🏅 입선\n[Crewing] 연합동아리 정보 플랫폼\n서비스 제작자: 동학대학운동(...,596,https://spartacodingclub.kr/blog/all-in-challe...
5,5,🏅 입선\n[학교생활 매니저] 학교생활 관리 서비스\n서비스 제작자: 아이칼F4(조...,837,https://spartacodingclub.kr/blog/all-in-challe...
6,6,🏅 입선\n[BLOTIE] 교내 외국인X내국인 매칭 및 교류 플랫폼\n서비스 제작자...,906,https://spartacodingclub.kr/blog/all-in-challe...
7,7,"사용한 기술 스택\n-FE(프론트엔드): Spring Boot, MySQL, Spr...",545,https://spartacodingclub.kr/blog/all-in-challe...


In [14]:
print(df.loc[4, "content"])

🏅 입선
[Crewing] 연합동아리 정보 플랫폼
서비스 제작자: 동학대학운동(김민아, 임경진, 신은혜, 고수)
💡신입생인 S는 대학 생활을 알차게 보내고 싶어서 연합 동아리에 가입하려고 했지만 어떤 동아리가 자신에게 맞을지 고민이 많았습니다. 인터넷에 검색해보니 연합 동아리 후기는 대부분 여기저기 흩어져 있고, 신뢰할 만한 정보를 찾기 어려웠습니다.
<Crewing>은 대학생들이 다양한 연합 동아리에 쉽게 가입하고, 적절한 동아리를 찾을 수 있도록 지원해주는 아카이빙 플랫폼입니다. 회원가입 시 생년, 성별, 관심 분야를 입력하자 딱 맞는 동아리를 추천해주죠. 플랫폼 내에서 동아리의 리크루팅 과정인 서류 전형, 인터뷰 일정, 최종 결과 발표 등을 한 번에 해결할 수 있어요. 실제 동아리에 가입한 사람들의 솔직한 후기를 제공해주기 때문에 보다 정확한 정보를 얻을 수 있어요. Crewing은 신뢰할 수 있는 정보와 솔직한 후기를 제공해주기 때문에 효율적으로 내게 꼭 맞는 동아리를 선택할 수 있습니다.
사용한 기술 스택 
-FE(프론트엔드): Spring Boot, Redis, MySQL
-BE(백엔드): SwiftUI Framework, OAuth 2.0


좀 더 프로젝트 별로 나뉜 걸 볼 수 있다.

### 3. Embedding & Vector Store

* OpenAIEmbeddings   
OpenAI의 API를 활용하여 임베딩 벡터로 변환

* Chroma   
임베딩 벡터를 저장하기 위한 오픈소스 소프트웨어

In [25]:
default_vectorstore = Chroma.from_documents(
    documents=default_splits,
    embedding=OpenAIEmbeddings(api_key=os.environ["OPENAI_API_KEY"]),
    persist_directory="./chroma_default"
)
default_retriever = default_vectorstore.as_retriever(search_kwargs={"k": 4})

In [24]:
new_vectorstore = Chroma.from_documents(
    documents=new_splits,
    embedding=OpenAIEmbeddings(api_key=os.environ["OPENAI_API_KEY"]),
    persist_directory="./chroma_new"
)
new_retriever = new_vectorstore.as_retriever(search_kwargs={"k": 4})

#### prompt

* 기본 제공된 rag-prompt 사용
* format_docs : 추출된 docs들을 문자열로 연결하는 함수 (prompt에 넣기 위함)


In [21]:
def format_docs(docs):
    return "\n\n---\n\n".join(doc.page_content for doc in docs)

prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [64]:
def get_prompt(retriever, user_query):
    retrieved_docs = retriever.invoke(user_query)
    retrieved_docs = retrieved_docs[:4]
    user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_query})
    print(user_prompt)

    return user_prompt


In [18]:
user_query = "ALL-in 코딩 공모전 수상작들을 요약해줘."

In [36]:
def generate_with_openai(retriever, user_query):
    user_prompt = get_prompt(retriever, user_query)
    response = llm.invoke(user_prompt)
    display_prompt_flow(user_query, response.content)


In [37]:
generate_with_openai(new_retriever, user_query)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘. \nContext: 코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.\n<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.\n\n---\n\n사용한 기술 스택\n-FE(프론트엔드): Spring Boot, MySQL, Spring WebSocket\n-BE(백엔드): React Native, TanStack Query, Axios\n코딩 공모전 수상작은 대학생들의 팀프로젝트를 통해 만들어진 웹/앱 서비스입니다. 캠퍼스에서의 문제를 해결하자는 참가자들의 아이디어에서 시작되었죠. 누구나 세상에 선보이고 싶은 나만의 아이디어와 기초 코딩 기술만 활용한다면, 얼마든지 서비스를 만들 수 있습니다. 스파르타코딩클럽의 내일배움캠프에서는 비전공, 초보자도 웹/앱 개발자로 거듭날 수 있는 다양한 트랙이 준비돼 있습니다. 나만의 아이디어를 세상에 선보이고 싶은 누구나에게 열려 있으니 주저말고 도

In [38]:
generate_with_openai(default_retriever, user_query)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘. \nContext: 사용한 기술 스택\n-FE(프론트엔드): Spring Boot, MySQL, Spring WebSocket\n-BE(백엔드): React Native, TanStack Query, Axios\n코딩 공모전 수상작은 대학생들의 팀프로젝트를 통해 만들어진 웹/앱 서비스입니다. 캠퍼스에서의 문제를 해결하자는 참가자들의 아이디어에서 시작되었죠. 누구나 세상에 선보이고 싶은 나만의 아이디어와 기초 코딩 기술만 활용한다면, 얼마든지 서비스를 만들 수 있습니다. 스파르타코딩클럽의 내일배움캠프에서는 비전공, 초보자도 웹/앱 개발자로 거듭날 수 있는 다양한 트랙이 준비돼 있습니다. 나만의 아이디어를 세상에 선보이고 싶은 누구나에게 열려 있으니 주저말고 도전해 보세요.\n💡<All-in> 코딩 공모전에서 만든 다양한 서비스를 만나보고 싶다면?\n다양한 서비스와 기발한 아이디어가 모인 곳에 초대합니다. 참가자들의 문제 해결방법이 궁금하시다면 지금 바로 ‘All-in 공모전’에서 만나보세요!\n👉🏻\xa0공모전 결과물 보러가기\n누구나 큰일 낼 수 있어\n스파르타코딩클럽\n글 | 신수지 팀스파르타 에디터\n\n---\n\n코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 

default_retriever와 new_retriever의 결과를 보면 내용이 다른데, 일단 6개 요약은 당연히 문서가 4개만 뽑히므로 어렵고, 각자 검색된 project에 대해서만 요약하는 것을 볼 수 있다.

다른 질문들을 비교해보면서 해보고자 한다.

In [43]:
def test_two_vectorDB(user_query):
    generate_with_openai(default_retriever, user_query)
    generate_with_openai(new_retriever, user_query)

**Q. 에코 클래스룸의 기술 스택은 어떻게 되나요?**

default_retriever의 결과를 보면 [에코 클래스룸] 의 기술 스택이 아니라 그 위에 소개된 프로젝트의 기술 스택인 걸 볼 수 있다.   
chunk 단위가 project 단위가 아니기 때문에 LLM이 한 chunk에 있는 기술 스택이 해당 프로젝트의 기술스택이라고 이해하여 일어난 일이다.

   
new_retriever의 경우에는 정확하게 대답하고 있는 것을 볼 수 있다. project 단위로 chunk를 끊어줬기 때문이다.

In [62]:
test_two_vectorDB("에코 클래스룸의 기술 스택은 어떻게 되나요?")

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: 에코 클래스룸의 기술 스택은 어떻게 되나요? \nContext: 사용한 기술 스택 \n-FE(프론트엔드): React Native, Expo, Axios\n-BE(백엔드): Spring Boot, Spring Security, MySQL, SSE, FCM, JWT, Postman\n🎖️\xa0우수상\n[에코 클래스룸] 수업 실시간 소통 서비스\n서비스 제작자: This is 스파게티!!!(박지성, 김서원, 박범수)\n💡K교수는 항상 수업이 끝난 후 난이도가 적절했는지 궁금했습니다. “질문 있나요?”라는 말이 수업의 마무리였지만, 대부분의 학생은 답이 없었죠. 그저 고개를 끄덕이는 몇몇 학생들만 보일 뿐, 정말 이해한 것인지, 질문할 용기가 없는 것인지 알 수 없었습니다. 이때문에 학생들이 수업 내용을 제대로 이해하고 있는지 확인하기 어려웠습니다. 어느 날 강의가 끝나고 몇몇 학생들이 몰려와 어렵다고, 다시 설명해 달라고 요청했습니다. 그제야 K교수는 알게 되었어요. 학생들이 이해하지 못하고 있었지만, 질문을 하지 않았던 것이었습니다.\n다른 학생들의 시선이 부담스러워서, 나만 모르는 거라고 생각해서 질문하지 않는 학생들을 위해 만들어진 서비스가 바로 <에코 클래스룸>입니다. 교수와 학생 간의 소통을 혁신적으로 바꾸기 위해 만들어졌어요. 학생들이 굳이 손을 들고 질문하지 않아도, 에코 클래스룸을 통해 자신의 의견을 표현할 수 있습니다. 익명으로 수업 중 자신의 의견이나

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: 에코 클래스룸의 기술 스택은 어떻게 되나요? \nContext: 🎖️\xa0우수상\n[에코 클래스룸] 수업 실시간 소통 서비스\n서비스 제작자: This is 스파게티!!!(박지성, 김서원, 박범수)\n💡K교수는 항상 수업이 끝난 후 난이도가 적절했는지 궁금했습니다. “질문 있나요?”라는 말이 수업의 마무리였지만, 대부분의 학생은 답이 없었죠. 그저 고개를 끄덕이는 몇몇 학생들만 보일 뿐, 정말 이해한 것인지, 질문할 용기가 없는 것인지 알 수 없었습니다. 이때문에 학생들이 수업 내용을 제대로 이해하고 있는지 확인하기 어려웠습니다. 어느 날 강의가 끝나고 몇몇 학생들이 몰려와 어렵다고, 다시 설명해 달라고 요청했습니다. 그제야 K교수는 알게 되었어요. 학생들이 이해하지 못하고 있었지만, 질문을 하지 않았던 것이었습니다.\n다른 학생들의 시선이 부담스러워서, 나만 모르는 거라고 생각해서 질문하지 않는 학생들을 위해 만들어진 서비스가 바로 <에코 클래스룸>입니다. 교수와 학생 간의 소통을 혁신적으로 바꾸기 위해 만들어졌어요. 학생들이 굳이 손을 들고 질문하지 않아도, 에코 클래스룸을 통해 자신의 의견을 표현할 수 있습니다. 익명으로 수업 중 자신의 의견이나 질문을 쉽게 제출할 수 있어 교수님은 실시간으로 학생들의 이해도를 파악할 수 있습니다. 수업 중 또는 후에 교수자가 수업에 대한 피드백을 받을 수 있는 평가 기능과 학생의 이해도를 테스트 할 수 있는 퀴즈 생성 기능이

**Q. 대상**

In [63]:
generate_with_openai(new_retriever, "대상")

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: 대상 \nContext: 코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.\n<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.\n\n---\n\n사용한 기술 스택\n-FE(프론트엔드): Spring Boot, MySQL, Spring WebSocket\n-BE(백엔드): React Native, TanStack Query, Axios\n코딩 공모전 수상작은 대학생들의 팀프로젝트를 통해 만들어진 웹/앱 서비스입니다. 캠퍼스에서의 문제를 해결하자는 참가자들의 아이디어에서 시작되었죠. 누구나 세상에 선보이고 싶은 나만의 아이디어와 기초 코딩 기술만 활용한다면, 얼마든지 서비스를 만들 수 있습니다. 스파르타코딩클럽의 내일배움캠프에서는 비전공, 초보자도 웹/앱 개발자로 거듭날 수 있는 다양한 트랙이 준비돼 있습니다. 나만의 아이디어를 세상에 선보이고 싶은 누구나에게 열려 있으니 주저말고 도전해 보세요.\n💡<All-in> 코딩 공

"대상" 이라는 간단한 query에 new_retriever도 제대로 대상 프로젝트에 대해 대답하지 못하는 모습을 볼 수 있다. openai API msg를 보면 대상 프로젝트에 대한 문서 자체가 retriever에서 나오지 못한 걸 볼 수 있는데, 추측하건데 문서 길이에 비해 user_query가 너무 짧고, keyword 기반의 검색이 아니기 때문에 similarity를 비교하면서 의미를 찾지 못하는 것 같다.

따라서 keywords 기반의 검색도 같이 하는 hybrid search를 langchain을 통해 쉽게 구현했다.

In [54]:
!pip install rank_bm25

In [58]:
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

# ✅ 2. Sparse 기반: BM25Retriever
bm25_retriever = BM25Retriever.from_documents(new_splits)
bm25_retriever.k = 3

# ✅ 3. Hybrid: EnsembleRetriever
hybrid_retriever = EnsembleRetriever(
    retrievers=[new_retriever, bm25_retriever],
    weights=[0.5, 0.5],  # dense와 sparse 각각의 가중치 (튜닝 가능)
)

In [65]:
generate_with_openai(hybrid_retriever, "대상")

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: 대상 \nContext: 사용한 기술 스택\n-FE(프론트엔드): Spring Boot, MySQL, Spring WebSocket\n-BE(백엔드): React Native, TanStack Query, Axios\n코딩 공모전 수상작은 대학생들의 팀프로젝트를 통해 만들어진 웹/앱 서비스입니다. 캠퍼스에서의 문제를 해결하자는 참가자들의 아이디어에서 시작되었죠. 누구나 세상에 선보이고 싶은 나만의 아이디어와 기초 코딩 기술만 활용한다면, 얼마든지 서비스를 만들 수 있습니다. 스파르타코딩클럽의 내일배움캠프에서는 비전공, 초보자도 웹/앱 개발자로 거듭날 수 있는 다양한 트랙이 준비돼 있습니다. 나만의 아이디어를 세상에 선보이고 싶은 누구나에게 열려 있으니 주저말고 도전해 보세요.\n💡<All-in> 코딩 공모전에서 만든 다양한 서비스를 만나보고 싶다면?\n다양한 서비스와 기발한 아이디어가 모인 곳에 초대합니다. 참가자들의 문제 해결방법이 궁금하시다면 지금 바로 ‘All-in 공모전’에서 만나보세요!\n👉🏻\xa0공모전 결과물 보러가기\n누구나 큰일 낼 수 있어\n스파르타코딩클럽\n글 | 신수지 팀스파르타 에디터\n\n---\n\n🏆\xa0대상\n[Lexi Note] 언어공부 필기 웹 서비스\n서비스 제작자: 다나와(김다애, 박나경)\n💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의

대상 프로젝트에 정확하게 대답하는 것을 볼 수 있었다.